In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('../data/weather_classification_data.csv')
train.head(1)

In [ ]:
from auto_ml_kinder import pre_processing as pp

In [ ]:
train.columns = train.columns.str.lower().str.replace(' ', '_')

In [ ]:
train.head(1)

In [ ]:
label_encode:list[pp.PreLabelEncoderConfig] = []
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='cloud_cover',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cloud_cover.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='season',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.season.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='location',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.location.value_counts().keys())}))
    )
label_encode.append(pp.PreLabelEncoderConfig(
    column_name='weather_type',
    label_encoding=list({pp.LabelEncodingDict(key, idx + 0) for idx, key in enumerate(train.weather_type.value_counts().keys())}))
    )
# label_encode.append(pp.PreLabelEncoderConfig(
#     column_name='restaurant_type',
#     label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.restaurant_type.value_counts().keys())}))
#     )
# label_encode.append(pp.PreLabelEncoderConfig(
#     column_name='cuisine',
#     label_encoding=list({pp.LabelEncodingDict(key, idx + 1) for idx, key in enumerate(train.cuisine.value_counts().keys())}))
#     )

numeric_col_changer:list[pp.PreNumericColDataChangeConfig] = []
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='facebook_popularity_quotient',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='instagram_popularity_quotient',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='overall_restaurant_rating',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='live_music_rating',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='ambience',data_type=int))
# numeric_col_changer.append(pp.PreNumericColDataChangeConfig(col_name='resturant_tier',data_type=int))

model_config = pp.PreProcessingConfig(
    encoding_dummies=[],
    exclude_columns=[],
    label_encode=label_encode,
    numeric_cols_data_changer=numeric_col_changer,
    target_column='weather_type',
    is_classification=True
)

In [ ]:
pre_processed_df = pp.process(train,model_config)

In [ ]:
pre_processed_df.head()

In [ ]:
from auto_ml_kinder import model_training_data_prep as dp
from auto_ml_kinder import model_training_helper as mth
from auto_ml_kinder import model_list_helper as mlh

In [ ]:
data = dp.ModelTrainingData(pre_processed_df, dp.ScalerType.STANDARD_SCALER, .95,use_pca=False,use_polynomials=False,use_feature_selection=False,create_clustering_feature_and_no_of_clusters=(False,3),is_classification=True)

In [ ]:
import keras as k
k.utils.to_categorical(data.Y_test_neural_network)

In [ ]:
trainer = mth.ModelTrainer(data=data)

In [ ]:
trainer.perform_operation_classification(exclude_models=[
    mlh.ModelAndParamClassifiction.SVC_Classification,
    mlh.ModelAndParamClassifiction.DecisionTree_Classifiction,
    # mlh.ModelAndParamClassifiction.RandomForest_Classifiction,
    mlh.ModelAndParamClassifiction.GradientBoosting_Classifiction,
    mlh.ModelAndParamClassifiction.KNeighbors_Classifiction
    ],permutate_n_less_column=0)
trainer.performance_df

In [ ]:
data = dp.ModelTrainingData(pre_processed_df, dp.ScalerType.QUANTILE_TRANSFORMER, .95,use_pca=False,use_polynomials=False,use_feature_selection=False,create_clustering_feature_and_no_of_clusters=(False,3))

In [ ]:
trainer.data = data

In [ ]:
trainer.perform_operation_classification(exclude_models=[
    mlh.ModelAndParamClassifiction.SVC_Classification,
    mlh.ModelAndParamClassifiction.DecisionTree_Classifiction,
    mlh.ModelAndParamClassifiction.RandomForest_Classifiction,
    mlh.ModelAndParamClassifiction.GradientBoosting_Classifiction,
    mlh.ModelAndParamClassifiction.KNeighbors_Classifiction,
    mlh.ModelAndParamClassifiction.Ridge_Classifiction
    ],permutate_n_less_column=2)
trainer.performance_df

In [ ]:
trainer.performance_df.loc[trainer.performance_df['score'].idxmax()]

In [ ]:
trainer.perform_neural_network_classification(totalExperiments=4,
                                          params=mth.NeuralNetwork_BayesianOptimization_Params(
                                              neurons_min_max=(128,128)
                                              ,batch_size_min_max=(32,32)
                                              ,dropout_rate_min_max=(.3,.7)
                                              ,epochs_min_max=(50,100)
                                              ,hidden_layers_min_max=(2,4)
                                              ,learning_rate_min_max=(.001,.1)
                                              ,normalization_min_max=(0,0)
                                              ,dropout_min_max=(1,1)
                                              ,activation_min_max=(0,3)
                                              ))

In [ ]:
trainer.neural_network_best_model_classification(epochs=10)

In [ ]:
trainer.performance_df

In [ ]:
import numpy as np
np.argmax(data.Y_test_neural_network, axis=1)

In [ ]:
data.Y_test_neural_network